https://medium.com/@Sean_Hsu/concurrency-parallelism-in-python-ebdc040e0881

### Tranditional multiprocessing and threading

In [11]:
# concurency package
import threading as th
import multiprocessing as mp
import concurrent.futures

import datetime
import functools
import time
import requests
import os


# use multiprocessing and threading


In [12]:
# cal cost time
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer


In [13]:
# Fibonacci
def fib(n):
    if n < 2:
        return 1
    return fib(n - 1) + fib(n - 2)


In [14]:
# do fib and show some info about thread and process
def do_some_fib(n, start):
    finish = fib(n)
    print(f'{n} ', th.current_thread().name)
    print(f'{n} pid:', os.getpid())
    print(f'{n} parent id:', os.getppid())
    print(f'Complete {n} levels FIB. Answer is {finish}. Cost time {datetime.datetime.now() - start}\n')
    
    

In [15]:
FIBS = [35, 28, 1, 25, 10]
@timer
def threading_fib():
    threads = []
    for i in range(len(FIBS)):
        threads.append(th.Thread(target=do_some_fib, args=(FIBS[i], datetime.datetime.now())))
        threads[i].start()

    for i in range(len(FIBS)):
        threads[i].join()
        
@timer
def processing_fib():
    processes = []
    for i in range(len(FIBS)):
        processes.append(mp.Process(target=do_some_fib, args=(FIBS[i], datetime.datetime.now())))
        processes[i].start()  
    for i in range(len(FIBS)):
        processes[i].join()
        
@timer    
def sync_fib():
    for i in FIBS:
        do_some_fib(i, datetime.datetime.now())
        
        

In [16]:
sync_fib()

35  MainThread
35 pid: 46897
35 parent id: 30062
Complete 35 levels FIB. Answer is 14930352. Cost time 0:00:02.941418

28  MainThread
28 pid: 46897
28 parent id: 30062
Complete 28 levels FIB. Answer is 514229. Cost time 0:00:00.106246

1  MainThread
1 pid: 46897
1 parent id: 30062
Complete 1 levels FIB. Answer is 1. Cost time 0:00:00.000111

25  MainThread
25 pid: 46897
25 parent id: 30062
Complete 25 levels FIB. Answer is 121393. Cost time 0:00:00.025721

10  MainThread
10 pid: 46897
10 parent id: 30062
Complete 10 levels FIB. Answer is 89. Cost time 0:00:00.000117

Finished 'sync_fib' in 3.0736 secs


In [17]:
threading_fib()
print('-' * 50)
processing_fib()

1 10   Thread-6
Thread-81 pid:
 10 pid: 46897
10 parent id: 30062
Complete 10 levels FIB. Answer is 89. Cost time 0:00:00.050495

25 46897
 1 parent id: 30062
Thread-7Complete 1 levels FIB. Answer is 1. Cost time 0:00:00.131751


25 pid: 46897
25 parent id:28   Thread-530062
Complete 25 levels FIB. Answer is 121393. Cost time 0:00:00.214786


28 pid: 46897
28 parent id: 30062
Complete 28 levels FIB. Answer is 514229. Cost time 0:00:00.288400

35  Thread-4
35 pid: 46897
35 parent id: 30062
Complete 35 levels FIB. Answer is 14930352. Cost time 0:00:03.107867

Finished 'threading_fib' in 3.1079 secs
--------------------------------------------------
1  MainThread
1 pid: 47379
1 parent id: 46897
Complete 1 levels FIB. Answer is 1. Cost time 0:00:00.013147
10  MainThread

10 pid: 47381
10 parent id: 46897
Complete 10 levels FIB. Answer is 89. Cost time 0:00:00.015323

25  MainThread
25 pid: 47380
25 parent id: 46897
Complete 25 levels FIB. Answer is 121393. Cost time 0:00:00.047050

28  Mai

### concurrent

In [18]:
URLS = [
    'https://docs.python.org/3/library/ast.html',
    'https://docs.python.org/3/library/abc.html',
    'https://docs.python.org/3/library/time.html',
    'https://docs.python.org/3/library/os.html',
    'https://docs.python.org/3/library/sys.html',
    'https://docs.python.org/3/library/io.html',
    'https://docs.python.org/3/library/pdb.html',
    'https://docs.python.org/3/library/weakref.html'
]

In [23]:
# use cocurrent

# cal cost time
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def get_content(url):
    print(url)
    print(f'{url} | ', th.current_thread().name, f' | pid:', os.getpid(), f' | parent id:', os.getppid())
    return requests.get(url).text

@timer
def thread_scrap():
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(get_content, url): url for url in URLS}
        print(future_to_url)
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
            except Execption as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                print('%r page length is %d' % (url, len(data)))

@timer
def processing_scrap():
    with concurrent.futures.ProcessPoolExecutor(10) as executor:
        future_to_url = {}
        for key, value in zip(URLS, executor.map(get_content, URLS)):
            future_to_url[key] = value
            print('%r page length is %d' % (key, len(value)))

                
@timer
def main():
    for url in URLS:
        try:
            data = get_content(url)
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page length is %d' % (url, len(data)))

In [24]:
main()


https://docs.python.org/3/library/ast.html
https://docs.python.org/3/library/ast.html |  MainThread  | pid: 46897  | parent id: 30062
Finished 'get_content' in 0.0779 secs
'https://docs.python.org/3/library/ast.html' page length is 51496
https://docs.python.org/3/library/abc.html
https://docs.python.org/3/library/abc.html |  MainThread  | pid: 46897  | parent id: 30062
Finished 'get_content' in 0.0789 secs
'https://docs.python.org/3/library/abc.html' page length is 42162
https://docs.python.org/3/library/time.html
https://docs.python.org/3/library/time.html |  MainThread  | pid: 46897  | parent id: 30062
Finished 'get_content' in 0.0843 secs
'https://docs.python.org/3/library/time.html' page length is 93380
https://docs.python.org/3/library/os.html
https://docs.python.org/3/library/os.html |  MainThread  | pid: 46897  | parent id: 30062
Finished 'get_content' in 0.0929 secs
'https://docs.python.org/3/library/os.html' page length is 495892
https://docs.python.org/3/library/sys.html
http

In [26]:
thread_scrap()
print('-' * 50)
processing_scrap()

https://docs.python.org/3/library/ast.htmlhttps://docs.python.org/3/library/abc.html
https://docs.python.org/3/library/ast.html |  ThreadPoolExecutor-2_0  | pid: 46897  | parent id: 30062
https://docs.python.org/3/library/time.html
https://docs.python.org/3/library/abc.html |  ThreadPoolExecutor-2_1 
https://docs.python.org/3/library/time.html |  ThreadPoolExecutor-2_2https://docs.python.org/3/library/os.html
https://docs.python.org/3/library/os.html |  ThreadPoolExecutor-2_3  | pid: 46897  | parent id: 30062
 | pid:  https://docs.python.org/3/library/sys.html | pid: 46897  | parent id: 30062

https://docs.python.org/3/library/sys.html |  ThreadPoolExecutor-2_4  | pid: 46897  | parent id: 30062
https://docs.python.org/3/library/io.htmlhttps://docs.python.org/3/library/pdb.html46897
https://docs.python.org/3/library/weakref.html

https://docs.python.org/3/library/weakref.html |  ThreadPoolExecutor-2_7  | pid: 46897  | parent id:{<Future at 0x7f99e86a5390 state=running>: 'https://docs.py

### The multiprocessing Module
#### 1 The Basics: Processes and Locks

https://asyncfor.com/posts/python-parallel-system-tools-pp4e.html

In [27]:
"""
multiprocess basics: Process works like threading.Thread, but 
runs function call in parallel in a process instead of a thread;
locks can be used to synchronize, e.g. prints on some platforms;
starts new interpreter on windows, forks a new process on unix;
"""

import os
from multiprocessing import Process, Lock

def whoami(label, lock):
    msg = '%s: name:%s, pid:%s'
    with lock:
        print(msg % (label, __name__, os.getpid()))

if __name__ == '__main__':
    lock = Lock()
    whoami('function call', lock)

    p = Process(target=whoami, args=('spawned child', lock))
    p.start()
    p.join()

    for i in range(5):
        Process(target=whoami, args=(('run process %s' % i), lock)).start()

    with lock:
        print('Main process exit.')

function call: name:__main__, pid:46897
spawned child: name:__main__, pid:47417
run process 0: name:__main__, pid:47418
run process 1: name:__main__, pid:47419
run process 2: name:__main__, pid:47420
run process 3: name:__main__, pid:47421
Main process exit.
run process 4: name:__main__, pid:47422
